In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [375]:
def delete_index(T, index):
    return torch.cat([T[:i], T[i+1:]])

In [376]:
class MomentumObject:
    def __init__(self, w, h):
        self.w, self.h = w,h
        self.position = np.array([w/2, h/2])
        self.blank_state = np.zeros((w,h))
        self.speed = np.random.randint(-2, 2, size=2)
    def _jump_bound(self, position):
        if position[0] >= self.w:
            position[0] = position[0] - self.w
        elif position[0] < 0:
            position[0] = self.w + position[0]
        if position[1] >= self.h:
            position[1] = position[1] - self.w
        elif position[1] < 0:
            position[1] = self.h + position[1]
    def get_state_action(self):
        return np.concatenate((self.position, self.speed))
    def step(self):
        self.position += self.speed
        self._jump_bound(self.position)
        self.speed = np.random.randint(-2, 2, size=2)
    def render(self, predict_position=None):
        state = self.blank_state.copy()
        position = np.round(self.position).astype(int)
        state[position[0], position[1]] = 1
        if predict_position is not None:
            pred_x = np.round(predict_position[0]*(self.w-1)).astype(int)
            pred_y = np.round(predict_position[1]*(self.h-1)).astype(int)
            state[pred_x, pred_y] = 0.2
        return state
w,h=10,10
momentum_object = MomentumObject(w, h)
momentum_object.render()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [377]:
num_buffer = 64
dimension_state = 2
num_state = momentum_object.position.shape[0] + momentum_object.speed.shape[0]
states_actions = torch.zeros((num_buffer, num_state))

def update_state_buffer(states_actions, remove_index=None):
    new_state_action = torch.unsqueeze(torch.tensor(momentum_object.get_state_action()), 0)
    if remove_index is None:
        states_actions = states_actions[:-1]
    else:
        states_actions = delete_index(states_actions, remove_index)
    return torch.cat((new_state_action, states_actions))
for i in range(num_buffer):
    momentum_object.step()
    states_actions=update_state_buffer(states_actions)
states_actions

tensor([[ 2.,  1., -2., -1.],
        [ 4.,  0., -2.,  1.],
        [ 5.,  9., -1.,  1.],
        [ 6.,  0., -1., -1.],
        [ 5.,  2.,  1., -2.],
        [ 6.,  2., -1.,  0.],
        [ 7.,  1., -1.,  1.],
        [ 7.,  3.,  0., -2.],
        [ 8.,  4., -1., -1.],
        [ 8.,  5.,  0., -1.],
        [ 7.,  6.,  1., -1.],
        [ 8.,  6., -1.,  0.],
        [ 7.,  7.,  1., -1.],
        [ 7.,  9.,  0., -2.],
        [ 7.,  1.,  0., -2.],
        [ 6.,  3.,  1., -2.],
        [ 5.,  4.,  1., -1.],
        [ 4.,  5.,  1., -1.],
        [ 5.,  4., -1.,  1.],
        [ 4.,  6.,  1., -2.],
        [ 3.,  6.,  1.,  0.],
        [ 2.,  7.,  1., -1.],
        [ 4.,  8., -2., -1.],
        [ 5.,  8., -1.,  0.],
        [ 4.,  0.,  1., -2.],
        [ 3.,  2.,  1., -2.],
        [ 4.,  3., -1., -1.],
        [ 4.,  3.,  0.,  0.],
        [ 5.,  4., -1., -1.],
        [ 4.,  5.,  1., -1.],
        [ 4.,  7.,  0., -2.],
        [ 4.,  6.,  0.,  1.],
        [ 5.,  6., -1.,  0.],
        [ 

In [385]:
def train(i, encoder, states_actions, optimizer_encoder):
    encoder.train()
    states_actions = states_actions.to(device)
    optimizer_encoder.zero_grad()
    
    current_state = torch.clone(states_actions[1:])
    next_state = torch.clone(states_actions[:-1, :2])
    
    next_state[:, 0] = next_state[:, 0]/(w-1)
    next_state[:, 1] = next_state[:, 1]/(h-1)
    
    state_prediction = encoder(current_state)
    loss = mse_loss(state_prediction, next_state)
    print("loss[", i,"]:", loss.mean())
    
    loss.mean().backward()
    optimizer_encoder.step()
    
    return torch.argmin(loss.mean(axis=1)).type(torch.int)
    
torch.manual_seed(1)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder = nn.Sequential(
                nn.Linear(4,16),
                nn.ReLU(),
                nn.Linear(16,32),
                nn.ReLU(),
                nn.Linear(32,8),
                nn.ReLU(),
                nn.Linear(8,2),
                nn.Sigmoid()).to(device).to(torch.float64)

optimizer_encoder = optim.Adam(encoder.parameters(), lr=0.001)
mse_loss = nn.MSELoss(reduction='none')

lowest_loss_index = None
for i in range(10000):
    momentum_object.step()
    states_actions=update_state_buffer(states_actions, lowest_loss_index)
    lowest_loss_index = train(i, encoder, states_actions, optimizer_encoder)


loss[ 0 ]: tensor(0.1102, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 1 ]: tensor(0.1097, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 2 ]: tensor(0.1092, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 3 ]: tensor(0.1087, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 4 ]: tensor(0.1083, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 5 ]: tensor(0.1078, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 6 ]: tensor(0.1074, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 7 ]: tensor(0.1070, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 8 ]: tensor(0.1066, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9 ]: tensor(0.1062, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 10 ]: tensor(0.1058, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 11 ]: tensor(0.1055, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 12 ]: tensor(0.1051, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 13 ]: tensor(0.1048, dtype=torch.float64, grad_fn=<Mean

loss[ 123 ]: tensor(0.0677, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 124 ]: tensor(0.0675, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 125 ]: tensor(0.0672, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 126 ]: tensor(0.0670, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 127 ]: tensor(0.0668, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 128 ]: tensor(0.0667, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 129 ]: tensor(0.0665, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 130 ]: tensor(0.0663, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 131 ]: tensor(0.0661, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 132 ]: tensor(0.0660, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 133 ]: tensor(0.0658, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 134 ]: tensor(0.0655, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 135 ]: tensor(0.0653, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 136 ]: tensor(0.0651, dtype=torc

loss[ 237 ]: tensor(0.0451, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 238 ]: tensor(0.0450, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 239 ]: tensor(0.0448, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 240 ]: tensor(0.0447, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 241 ]: tensor(0.0446, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 242 ]: tensor(0.0445, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 243 ]: tensor(0.0444, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 244 ]: tensor(0.0443, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 245 ]: tensor(0.0441, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 246 ]: tensor(0.0440, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 247 ]: tensor(0.0439, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 248 ]: tensor(0.0437, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 249 ]: tensor(0.0436, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 250 ]: tensor(0.0435, dtype=torc

loss[ 348 ]: tensor(0.0285, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 349 ]: tensor(0.0284, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 350 ]: tensor(0.0283, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 351 ]: tensor(0.0282, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 352 ]: tensor(0.0281, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 353 ]: tensor(0.0280, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 354 ]: tensor(0.0279, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 355 ]: tensor(0.0278, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 356 ]: tensor(0.0277, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 357 ]: tensor(0.0276, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 358 ]: tensor(0.0275, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 359 ]: tensor(0.0274, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 360 ]: tensor(0.0273, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 361 ]: tensor(0.0272, dtype=torc

loss[ 460 ]: tensor(0.0230, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 461 ]: tensor(0.0230, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 462 ]: tensor(0.0230, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 463 ]: tensor(0.0230, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 464 ]: tensor(0.0229, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 465 ]: tensor(0.0229, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 466 ]: tensor(0.0229, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 467 ]: tensor(0.0229, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 468 ]: tensor(0.0229, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 469 ]: tensor(0.0228, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 470 ]: tensor(0.0228, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 471 ]: tensor(0.0228, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 472 ]: tensor(0.0228, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 473 ]: tensor(0.0228, dtype=torc

...

loss[ 9527 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9528 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9529 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9530 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9531 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9532 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9533 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9534 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9535 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9536 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9537 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9538 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9539 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9540 ]: tensor(0.00

loss[ 9642 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9643 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9644 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9645 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9646 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9647 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9648 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9649 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9650 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9651 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9652 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9653 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9654 ]: tensor(0.0007, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9655 ]: tensor(0.00

loss[ 9753 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9754 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9755 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9756 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9757 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9758 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9759 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9760 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9761 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9762 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9763 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9764 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9765 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9766 ]: tensor(0.00

loss[ 9867 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9868 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9869 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9870 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9871 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9872 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9873 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9874 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9875 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9876 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9877 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9878 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9879 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9880 ]: tensor(0.00

loss[ 9981 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9982 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9983 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9984 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9985 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9986 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9987 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9988 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9989 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9990 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9991 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9992 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9993 ]: tensor(0.0006, dtype=torch.float64, grad_fn=<MeanBackward0>)
loss[ 9994 ]: tensor(0.00

In [387]:
fig = plt.figure()
im = plt.imshow(momentum_object.render(), animated=True)
def updatefig(*args):
    position = torch.tensor([momentum_object.get_state_action()])
    predict_position = encoder(position).detach().numpy()[0]
    momentum_object.step()
    im.set_array(momentum_object.render(predict_position))
ani = FuncAnimation(fig, updatefig, interval=500, blit=True)
# plt.show()

<IPython.core.display.Javascript object>

In [180]:
for i in range(100):
    momentum_object.step()
    position = torch.tensor([momentum_object.get_state_action()])
    predict_position = encoder(position).detach().numpy()[0]
    print(predict_position)
    im.set_array(momentum_object.render(predict_position))

[0.77829137 0.33494916]
[0.99617782 0.66948888]
[0.11137521 0.9982514 ]
[0.33371092 0.22304768]
[0.55515901 0.55713124]
[0.77784401 0.88946347]
[0.998113   0.11263267]
[0.11167683 0.44720368]
[0.33348316 0.77863872]
[0.55480169 0.00562267]
[0.77829137 0.33494916]
[0.99617782 0.66948888]
[0.11137521 0.9982514 ]
[0.33371092 0.22304768]
[0.55515901 0.55713124]
[0.77784401 0.88946347]
[0.998113   0.11263267]
[0.11167683 0.44720368]
[0.33348316 0.77863872]
[0.55480169 0.00562267]
[0.77829137 0.33494916]
[0.99617782 0.66948888]
[0.11137521 0.9982514 ]
[0.33371092 0.22304768]
[0.55515901 0.55713124]
[0.77784401 0.88946347]
[0.998113   0.11263267]
[0.11167683 0.44720368]
[0.33348316 0.77863872]
[0.55480169 0.00562267]
[0.77829137 0.33494916]
[0.99617782 0.66948888]
[0.11137521 0.9982514 ]
[0.33371092 0.22304768]
[0.55515901 0.55713124]
[0.77784401 0.88946347]
[0.998113   0.11263267]
[0.11167683 0.44720368]
[0.33348316 0.77863872]
[0.55480169 0.00562267]
[0.77829137 0.33494916]
[0.99617782 0.66